# Training RelateAnything with PSG

In [ ]:
%load_ext autoreload
%autoreload 2

from segment_anything import build_sam, SamPredictor, SamAutomaticMaskGenerator

In [ ]:
predictor = SamPredictor(build_sam(checkpoint="./segment_anything/checkpoints/sam_vit_h_4b8939.pth"))
mask_generator = SamAutomaticMaskGenerator(build_sam(checkpoint="./segment_anything/checkpoints/sam_vit_h_4b8939.pth"))

### Get PSG Dataset

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import pprint

import numpy as np

from openpsg.utils.vis_tools.datasets import coco_dir
from openpsg.utils.vis_tools.preprocess import load_json

from detectron2.data.detection_utils import read_image
from detectron2.utils.colormap import colormap
from panopticapi.utils import rgb2id

def get_colormap(num_colors: int):
    return (np.resize(colormap(), (num_colors, 3))).tolist()

In [ ]:
# set working path as home dir to easy access data
psg_dataset_file = load_json(Path("data/psg/psg.json"))
print('keys: ', list(psg_dataset_file.keys()))

psg_thing_cats = psg_dataset_file['thing_classes']
psg_stuff_cats = psg_dataset_file['stuff_classes']
psg_obj_cats = psg_thing_cats + psg_stuff_cats
psg_rel_cats = psg_dataset_file['predicate_classes']
psg_dataset = {d["image_id"]: d for d in psg_dataset_file['data']}
# psg_dataset_coco_id = {d["coco_image_id"]: d for d in psg_dataset_file['data']}

print('Number of images: {}'.format(len(psg_dataset)))
print('# Object Classes: {}'.format(len(psg_obj_cats)))
print('# Relation Classes: {}'.format(len(psg_rel_cats)))

In [ ]:
example_img_id = '2346957'
data = psg_dataset[example_img_id]
data['relations']

In [ ]:
# show origin image
image = read_image(coco_dir / data["file_name"], format="RGB")
seg_map = read_image(coco_dir / data["pan_seg_file_name"], format="RGB")

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(image)
axs[0].set_title("Original Image")
axs[1].imshow(seg_map)
axs[1].set_title("Segment Map")

# Hide the x and y axis labels for both subplots
for ax in axs:
    ax.set_xticklabels([])
    ax.set_yticklabels([])

# Display the subplots
plt.show()

# show seg annotations
seg_map = rgb2id(seg_map)
print('Segments IDs: ', np.unique(seg_map))
print('Segments Annotations: ')
pprint.pprint(data["segments_info"])

In [ ]:
# get seperate masks
gt_masks = []
labels_coco = []
for i, s in enumerate(data["segments_info"]):
    label = psg_obj_cats[s["category_id"]]
    labels_coco.append(label)
    gt_masks.append(seg_map == s["id"])

In [ ]:
from PIL import Image
pilimage = Image.open(coco_dir / data["file_name"])
width, height = pilimage.size
full_area = width * height

### Segment Everything

In this part, we have sam_feats as all masks features.

In [ ]:
sam_masks = mask_generator.generate(image)

In [ ]:
sam_masks[0]

In [ ]:
filtered_masks = [d for d in sam_masks if d['area'] > 0.03 * full_area]
print('number of masks left:', len(filtered_masks))

In [ ]:
def iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

def sort_and_deduplicate(sam_masks, iou_threshold=0.8):
    # Sort the sam_masks list based on the area value
    sorted_masks = sorted(sam_masks, key=lambda x: x['area'], reverse=True)

    # Deduplicate masks based on the given iou_threshold
    filtered_masks = []
    for mask in sorted_masks:
        duplicate = False
        for filtered_mask in filtered_masks:
            if iou(mask['segmentation'], filtered_mask['segmentation']) > iou_threshold:
                duplicate = True
                break

        if not duplicate:
            filtered_masks.append(mask)

    return filtered_masks

# Example usage
filtered_masks = sort_and_deduplicate(filtered_masks)
print('number of masks left:', len(filtered_masks))

In [ ]:
ddup_masks = sort_and_deduplicate(sam_masks)
print('number of deduplicated masks:', len(ddup_masks))

In [ ]:
def show_anns(anns, color='auto'):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        if color == 'auto':
            for i in range(3):
                img[:,:,i] = color_mask[i]
        else:
            for i in range(3):
                img[:,:,0] = 1
                img[:,:,1] = 0
                img[:,:,2] = 0
    return np.dstack((img, m*0.35))

In [ ]:
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', 
               s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', 
               s=marker_size, edgecolor='white', linewidth=1.25)   

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

In [ ]:
import math

# Calculate the number of rows needed
num_rows = math.ceil(len(ddup_masks) / 5)

# Create the subplots
fig, axs = plt.subplots(num_rows, 5, figsize=(15, 3*num_rows))

# Loop through the images and add them to the subplots
for i, mask in enumerate(ddup_masks):
    row = i // 5
    col = i % 5
    
    axs[row, col].imshow(img, alpha=0.3)
    axs[row, col].imshow(show_anns(ddup_masks[i:i+1], 'red'))
    axs[row, col].axis('off')
    axs[row, col].set_title(i)
    

# Display the subplots
plt.show()

### GT Matching

In [ ]:
gt_masks[0]

In [ ]:
gt_feats = []

for gt_mask in gt_masks:
    max_iou = 0
    best_feat = None

    for mask_dict in ddup_masks:
        current_iou = iou(gt_mask, mask_dict['segmentation'])

        if current_iou > max_iou:
            max_iou = current_iou
            best_feat = mask_dict['feat']

    gt_feats.append(best_feat)

gt_feats = np.array(gt_feats)

### Gather all data

In [ ]:
data['relations']

In [ ]:
len(gt_feats)

In [ ]:
list(psg_dataset.keys())[0]

In [ ]:
save_entry = {'id': id_num,
              'feat': gt_feats,
              'relations': data['relations'],
              'is_train': id_num in psg_dataset_file['test_image_ids'],
             }

### Set training data

In [ ]:
print('done')

In [ ]:
with np.load('feats/save_dict_2346957.npz', allow_pickle=True) as data:
    print(len(data['feat']))
    print(data['relations'])
    for i, token in enumerate(data['feat']):
        print(i)
        print(token)

In [ ]:
import glob
import numpy as np
from tqdm import tqdm

# List all npz files in the feats directory
file_list = glob.glob("feats/save_dict_*.npz")

train_data = []
train_label = []

test_data = []
test_label = []

for file in tqdm(file_list):
    with np.load(file, allow_pickle=True) as data:
        pairs_and_labels = []
        for relation in data['relations']:
            feat_id_1, feat_id_2, label = relation
            feature_1 = np.array(data['feat'][feat_id_1])
            feature_2 = np.array(data['feat'][feat_id_2])
            try:
                feature_pair = np.concatenate((feature_1, feature_2))
                if data['is_train']:
                    train_data.append(feature_pair)
                    train_label.append(label)
                else:
                    test_data.append(feature_pair)
                    test_label.append(label)
            except ValueError:
                print(file)

In [ ]:
train_data = np.array(train_data)
train_label = np.array(train_label)
print('train data size:', train_data.shape, 'label size: ', train_label.shape)

test_data = np.array(test_data)
test_label = np.array(test_label)
print('test data size:', test_data.shape, 'label size: ', test_label.shape)

In [ ]:
np.savez('relation_feat', 
         train_data=train_data, train_label=train_label, 
         test_data=test_data, test_label=test_label)

In [ ]:
len(test_data)

### Inference

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, dropout_prob=0.1):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

In [ ]:
# load model
input_size = 512
hidden_size = 256
num_classes = 56
model = MLP(input_size, hidden_size, num_classes)
model.load_state_dict(torch.load("best_model.pth"))

### Segment with Prompt

In this section, we extract the mask feature as feats.

In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

image_name = './images/dog.jpg'

image = cv2.imread(image_name)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

pilimage = Image.open(image_name)
width, height = pilimage.size
full_area = width * height

In [ ]:
predictor = SamPredictor(build_sam(checkpoint="./segment_anything/checkpoints/sam_vit_h_4b8939.pth"))

In [ ]:
predictor.set_image(image)

In [ ]:
# dog
input_point = np.array([[200, 375]])
input_label = np.array([1])

# bowl
# input_point = np.array([[600, 200]])
# input_label = np.array([1])

# human
# input_point = np.array([[700, 50]])
# input_label = np.array([1])

# ground
# input_point = np.array([[500, 400]])
# input_label = np.array([1])

plt.imshow(image)
show_points(input_point, input_label, plt.gca())
# plt.axis('off')
plt.show()

mask1, score1, logit1, feat1 = predictor.predict(point_coords=input_point,
                                                 point_labels=input_label,
                                                 multimask_output=False,)

In [ ]:
# dog
# input_point = np.array([[200, 375]])
# input_label = np.array([1])

# bowl
# input_point = np.array([[600, 200]])
# input_label = np.array([1])

# human
# input_point = np.array([[700, 50]])
# input_label = np.array([1])

# sofa
input_point = np.array([[150, 100]])
input_label = np.array([1])

# # ground
# input_point = np.array([[500, 400]])
# input_label = np.array([1])

plt.imshow(image)
show_points(input_point, input_label, plt.gca())
# plt.axis('off')
plt.show()

mask2, score2, logit2, feat2 = predictor.predict(point_coords=input_point,
                                                 point_labels=input_label,
                                                 multimask_output=False,)

In [ ]:
concat_input = torch.cat((feat1, feat2), dim=2)

In [ ]:
relation_classes = [
    'over', 'in front of', 'beside', 'on', 'in', 'attached to', 'hanging from', 'on back of', 'falling off', 
    'going down', 'painted on', 'walking on', 'running on', 'crossing', 'standing on', 'lying on', 'sitting on', 
    'flying over', 'jumping over', 'jumping from', 'wearing', 'holding', 'carrying', 'looking at', 'guiding', 
    'kissing', 'eating', 'drinking', 'feeding', 'biting', 'catching', 'picking', 'playing with', 'chasing', 
    'climbing', 'cleaning', 'playing', 'touching', 'pushing', 'pulling', 'opening', 'cooking', 'talking to', 
    'throwing', 'slicing', 'driving', 'riding', 'parked on', 'driving on', 'about to hit', 'kicking', 'swinging', 
    'entering', 'exiting', 'enclosing', 'leaning on']

In [ ]:
output = model(concat_input)
topk_indices = torch.argsort(-output).flatten()

In [ ]:
top3_rel = [relation_classes[indice] for indice in topk_indices][:5]

In [ ]:
top3_rel

In [ ]:
sigmoid_output = torch.sigmoid(output)
rounded_output = torch.round(sigmoid_output * 100) / 100
print(rounded_output)

In [ ]:
for i, (mask, score) in enumerate(zip(masks, scores)):
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    show_mask(mask, plt.gca())
    show_points(input_point, input_label, plt.gca())
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')
    plt.show() 